$$
HotSpots = Fechamento^{\alpha}*Volume^{\beta}*SENT^{\gamma}*Volatilidade^{\delta}*RSI^{\theta}*Trends^{\omega}
$$

- Sentimento deveria ser estimado em ln?

**Ideia**: selecionar períodos aleatórios de rebalanceamento : série binária com 1=(data teve rebalanceamento); repetir várias vezes -->    
Pegar as séries com melhores resultados --> rodar a regressão nessas séries e fazer média dos parâmetros ponderados por (1-pvalor)  
Resultado: índice com melhores momentos de rebalanceamento

**Ideia**: na hora de simular as carteiras com rebalançeamento em momentos aleatórios, a quantidade de rebalanceamentos (frequência) vai ser dada por uma distribuição (qui-quadrado? poisson?) --> depois, a gente vê a distribuição da frequência das carteiras mais eficientes pra saber apx a frequência ótima de rebalanceamentos --> depois roda com essa nova distribuição mais eficiente

In [102]:
import pandas as pd
import numpy as np
import os
import sqlite3
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [103]:
sent = pd.read_csv(r'C:\Users\loren\Documents\Programação\Github\quantamental-Dev_Wizards\data\sentimentos\noticias_sentimentos.csv')
sent['Data'] = pd.to_datetime(sent['Data'], format='%Y-%m-%d')
sent.loc[sent['Sentimento'] == 'POSITIVE', 'Score Signal'] = sent.loc[sent['Sentimento'] == 'POSITIVE', 'Score']
sent.loc[sent['Sentimento'] == 'NEGATIVE', 'Score Signal'] = sent.loc[sent['Sentimento'] == 'NEGATIVE', 'Score'] * -1
sent.loc[sent['Sentimento'] == 'NEUTRAL', 'Score Signal'] = 0

In [104]:
def indicador_sentimento(freq='D'):
    def agg_score(group):
        if group['Sentimento'].nunique() > 1:
            signal = group['Score Signal'].sum() / group['Score Modificado'].sum()
            return signal
        else:
            return group['Score Signal'].mean()

    sent['Score Modificado'] = np.where(sent['Sentimento'] == "NEUTRAL", 0.5, 1) * sent['Score']
    sentimentos_df = sent.resample(freq, on='Data').apply(agg_score).to_frame(name='Signal')
    return sentimentos_df

sent = indicador_sentimento()

In [105]:
# Pegando todos os dados

# SENTIMENTOS
sent = sent.reset_index()
sent.rename(columns={'Data': 'Date'}, inplace=True)
sent['Date'] = pd.to_datetime(sent['Date'])
sent = sent[(sent['Date'] >= '2014-09-08') & (sent['Date'] <= '2020-12-31')]

start = sent['Date'].min().strftime('%Y-%m-%d')
end = sent['Date'].max().strftime('%Y-%m-%d')
print(f'{start} - {end}')

# FECHAMENTO AJUSTADO
os.chdir(os.getcwd())
conn = sqlite3.connect(r"C:\databases\portfolio_quantamental.db")
cur = conn.cursor()

ibov_close = pd.read_sql('SELECT * FROM ibov_adj_close', conn)
ibov_close = ibov_close[(ibov_close['Date'] >= start) & (ibov_close['Date'] <= end)]
ibov_close['Date'] = pd.to_datetime(ibov_close['Date'])

# VIX
vix = pd.read_excel(r'C:\Users\loren\Documents\Programação\Github\quantamental-Dev_Wizards\data\vix\PerformanceGraphExport.xls')
vix.rename(columns={'Effective date ': 'Date'}, inplace=True)
vix['Date'] = pd.to_datetime(vix['Date'])

# RSI e VOLUME
rsi = pd.read_csv(r'C:\Users\loren\Documents\Programação\Github\quantamental-Dev_Wizards\data\momentum_vol\ibov_vol_rsi.csv')
rsi['Date'] = pd.to_datetime(rsi['Date'])
rsi = rsi[(rsi['Date'] >= start) & (rsi['Date'] <= end)]

# GOOGLE TRENDS
trends = pd.read_csv(r'C:\Users\loren\Documents\Programação\Github\quantamental-Dev_Wizards\data\trends\interesses_google.csv')
trends.rename(columns={'date': 'Date'}, inplace=True)
trends['Date'] = pd.to_datetime(trends['Date'])
trends = trends[(trends['Date'] >= start) & (trends['Date'] <= end)]

# REBALANCEAMENTOS
conn = sqlite3.connect(r"C:\databases\penalized_simulations.db")
cur = conn.cursor()
rebalanceamentos = pd.read_sql('SELECT * FROM simulated_portfolios', conn)
rebalanceamentos = rebalanceamentos.T

cur.close()

2014-09-08 - 2020-12-31


In [110]:
print(ibov_close.shape, vix.shape, rsi.shape, trends.shape, rebalanceamentos.shape)   

(1570, 89) (749, 2) (1558, 3) (75, 59) (1569, 30)
